In [1]:
# First import the libraries that we need to use
import os
import pandas as pd
import requests
import json
from dotenv import load_dotenv
load_dotenv()
"""
FYI:

I currently have this set in my .env file at the top of the project folder:

COINBASE_API_KEY=""
COINBASE_PRO_FQDN="https://api-pro.coinbase.com"
COINBASE_PUB_FQDN="https://api.exchange.coinbase.com"
COINBASE_SANDBOX_FQDN="https://api-public.sandbox.exchange.coinbase.com"

"""
# not needed
# cb_api = os.getenv("COINBASE_API_KEY")
cb_pro = os.getenv("COINBASE_PRO_FQDN")
cb_pub = os.getenv("COINBASE_PUB_FQDN")
cb_sandbox = os.getenv("COINBASE_SANDBOX_FQDN")

print(cb_pro)
print(cb_pub)
print(cb_sandbox)


https://api-pro.coinbase.com
https://api.exchange.coinbase.com
https://api-public.sandbox.exchange.coinbase.com


In [3]:
def get_currencies():
    """
    Returns DataFrame of currencies available on coinbase exchange.
    
    """
    coin_url = f"{cb_pub}/currencies"
    coin_response = requests.get(coin_url)
    if coin_response.status_code == 200:
        currencies_df = pd.DataFrame(json.loads(coin_response.text), columns=['id', 'name', 'min_size', 'status', 'max_precision']).fillna('')
        currencies_df.to_csv("coinbase_currencies.csv", index=False)
        display(currencies_df.head(5))
        display(currencies_df.tail(5))
    return

get_currencies()


,id,name,min_size,status,max_precision
0,CVC,Civic,1,online,0.00000001
1,OGN,Origin Token,0.1,online,0.01
2,TRAC,OriginTrail,0.1,online,0.0001
3,POWR,Power Ledger,0.000001,online,0.000001
4,ORN,Orion Protocol,0.1,online,0.01


,id,name,min_size,status,max_precision
173,ROSE,Oasis Network,0.00000001,online,0.00000001
174,YFII,DFI.Money,0.0001,online,0.00000001
175,UNI,Uniswap,1,online,0.000001
176,BNT,Bancor,1,online,0.000001
177,UST,TerraUSD,1,online,0.01


In [6]:
## Fetch daily candle data and save to CSV file

def fetch_daily_data(symbol):
    pair_split = symbol.split('/')  # symbol must be in format XXX/XXX ie. BTC/EUR
    symbol = pair_split[0] + '-' + pair_split[1]
    url = f'{cb_pub}/products/{symbol}/candles?granularity=86400'
    response = requests.get(url)
    if response.status_code == 200:
        data = pd.DataFrame(json.loads(response.text), columns=['unix', 'low', 'high', 'open', 'close', 'volume'])
        data['date'] = pd.to_datetime(data['unix'], unit='s')  # convert to a readable date
        data['vol_fiat'] = data['volume'] * data['close'] # multiply the BTC volume by closing price to approximate fiat volume

        if data is None:
            print("Did not return any data from Coinbase for this symbol")
        else:
            filename = f'coinbase_dailydata_{pair_split[0] + pair_split[1]}.csv'
            data.to_csv(filename, index=False)
            print(f'Created CSV file: {filename}.')
    else:
        print("Did not receieve OK response from Coinbase API")

if __name__ == "__main__":
    pair = "ADA/USD"    # <======== STATICALLY setting this for now
    fetch_daily_data(symbol=pair)


Created CSV file: coinbase_dailydata_ADAUSD.csv.


In [38]:
## Get product information
product_url = f"{cb_pub}/products"
headers = {"Accept": "application/json"}
product_response = requests.get(product_url, headers=headers)
product_columns=["id","base_currency","quote_currency","base_min_size","base_max_size","quote_increment","base_increment","display_name","min_market_funds","max_market_funds","margin_enabled","fx_stablecoin","max_slippage_percentage","post_only","limit_only","cancel_only","trading_disabled","status","status_message","auction_mode"]
product_df = pd.DataFrame(json.loads(product_response.text), columns=product_columns)
## We should save this data to local CSV and dataframe
display(product_df)
# display(product_df.tail(5))

,id,base_currency,quote_currency,base_min_size,base_max_size,quote_increment,base_increment,display_name,min_market_funds,max_market_funds,margin_enabled,fx_stablecoin,max_slippage_percentage,post_only,limit_only,cancel_only,trading_disabled,status,status_message,auction_mode
0,XYO-EUR,XYO,EUR,17,7100000,0.000001,0.1,XYO/EUR,0.84,170000,False,False,0.03000000,False,False,False,False,online,,False
1,UPI-USD,UPI,USD,10,8200000,0.00001,1,UPI/USD,1,100000,False,False,0.03000000,False,False,False,False,online,,False
2,POWR-EUR,POWR,EUR,1.3,70000,0.0001,0.1,POWR/EUR,0.84,43000,False,False,0.03000000,False,True,False,False,online,,False
3,RLC-USD,RLC,USD,0.22,140000,0.01,0.01,RLC/USD,1,360000,False,False,0.03000000,False,False,False,False,online,,False
4,APE-EUR,APE,EUR,0.05,63000,0.001,0.01,APE/EUR,1,250000,False,False,0.03000000,False,True,False,False,online,,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
470,XRP-GBP,XRP,GBP,1,500000,0.0001,0.000001,XRP/GBP,10,100000,False,False,0.10000000,False,False,False,True,delisted,,False
471,XRP-USD,XRP,USD,1,500000,0.0001,0.000001,XRP/USD,10,100000,False,False,0.10000000,False,False,False,True,delisted,,False
472,GNT-USDC,GNT,USDC,1,490000,0.000001,1,GNT/USDC,1,200000,False,False,0.03000000,False,False,False,True,delisted,,False
473,XRP-EUR,XRP,EUR,1,500000,0.0001,0.000001,XRP/EUR,10,100000,False,False,0.10000000,False,False,False,True,delisted,,False


In [40]:
## Get trades for ...

def fetch_trade_data(symbol):
    pair_split = symbol.split('/')
    symbol = pair_split[0] + '-' + pair_split[1]
    trades_url = f"{cb_pub}/products/{symbol}/trades"
    headers = {"Accept": "application/json"}
    trades_columns = ["time","trade_id","price","size","side"]
    response = requests.get(trades_url, headers=headers)

    if response.status_code == 200:
        trades_df = pd.DataFrame(json.loads(response.text), columns=trades_columns)
        trades_df['date'] = pd.to_datetime(trades_df['time'])  # convert to a readable date

        if trades_df is None:
            print("Did not return any data from Coinbase for this symbol")
        else:
            filename = f'coinbase_trades_{pair_split[0] + pair_split[1]}.csv'
            trades_df.to_csv(filename, index=False)
            print(f'Created CSV file: {filename}.')
    else:
        print("Did not receieve OK response from Coinbase API")

if __name__ == "__main__":
    pair = "ADA/USD"
    fetch_trade_data(symbol=pair)


Created CSV file: coinbase_trades_ADAUSD.csv.


In [39]:
for id in product_df['id']:
    print(id)
"""
The coinbase sandbox endpoint has only 10 products

BTC-EUR
BAT-USDC
LINK-USDC
ETH-BTC
BTC-USD
LINK-USD
BTC-GBP
XRP-USD
XRP-EUR
XRP-BTC
XRP-GBP

The pro endpoint has 475 products
"""

XYO-EUR
UPI-USD
POWR-EUR
RLC-USD
APE-EUR
SHIB-EUR
CVC-USD
CLV-EUR
TRB-BTC
DASH-BTC
ENJ-USDT
RLY-EUR
RGT-USD
AVAX-EUR
PLU-USD
MDT-USD
GMT-USDT
ICP-USD
DOT-BTC
MPL-USD
MATIC-USD
ZRX-EUR
RAD-BTC
BAL-BTC
APE-USD
FORTH-USD
EOS-BTC
UST-USDT
BAND-EUR
XLM-USD
TRU-USDT
MLN-USD
COVAL-USD
NMR-GBP
RNDR-EUR
WCFG-USDT
FIL-GBP
RNDR-USDT
IDEX-USD
MASK-EUR
RAD-USD
ETC-EUR
UST-EUR
KRL-EUR
MANA-EUR
ETH-GBP
XLM-BTC
WCFG-USD
ORN-BTC
REP-USD
DOGE-USD
MUSD-USD
LTC-BTC
REQ-EUR
XLM-USDT
BTRST-EUR
YFI-BTC
BCH-GBP
AIOZ-USDT
SUPER-USDT
AXS-EUR
GMT-USD
ANKR-GBP
DESO-USD
AAVE-EUR
NKN-GBP
REP-BTC
MIR-EUR
BADGER-USDT
REQ-USDT
ICP-BTC
BAT-BTC
BUSD-USD
BTC-EUR
RNDR-USD
POLS-USDT
RAD-GBP
STX-USD
DOT-USDT
OMG-BTC
ZEN-USD
GST-USD
BTRST-USD
MASK-USD
GALA-EUR
TRAC-EUR
LINK-ETH
SOL-USD
WCFG-BTC
CGLD-EUR
AAVE-GBP
OGN-BTC
BTC-UST
CGLD-BTC
UST-USD
TRB-USD
AUCTION-EUR
BNT-EUR
SOL-GBP
ALCX-USDT
DOT-EUR
NU-EUR
BAL-USD
PERP-EUR
COTI-USD
CRPT-USD
SPELL-USD
FARM-USD
BLZ-USD
BCH-USD
SOL-BTC
SNX-EUR
DDX-EUR
AAVE-BTC
FET-USD
ASM-USD
BAN

'\nThe coinbase sandbox endpoint has only 10 products\nBTC-EUR\nBAT-USDC\nLINK-USDC\nETH-BTC\nBTC-USD\nLINK-USD\nBTC-GBP\nXRP-USD\nXRP-EUR\nXRP-BTC\nXRP-GBP\n\nThe pro endpoint has 475 products\n'